# Comparing Language as Vectors with Word2Vec and Cosine Similarity

For this problem, you'll use a popular technique called Word2Vec that represents words as vectors in a high-dimensional space. The key idea is that words that are used in similar ways will be close to each other in this space. You can read the original paper by Mikolov et al. [here](https://arxiv.org/pdf/1301.3781.pdf).

## How it works

Word2Vec uses a shallow neural network to learn word associations from a large corpus of text. It has two main architectures for training: Continuous Bag of Words (CBOW) and Skip-Gram.

- **Continuous Bag of Words (CBOW):** This model predicts a target word based on its surrounding context words. For example, given the sentence "the cat sat on the mat," CBOW would use the words "the," "sat," "on," and "the" to predict "cat".
- **Skip-Gram:** This model works in the reverse manner of CBOW. It uses a target word to predict its surrounding context words. Using the same sentence, Skip-Gram would take "cat" as input to predict "the," "sat," "on," and "the".

Both models rely on the distributional hypothesis, which suggests that words appearing in similar contexts tend to have similar meanings. Through training, Word2Vec adjusts the vectors so that words with similar meanings are positioned close to each other in the vector space.

## Using it for language comparison

Of course, we don't need to know the exact details of how it works. We can use pre-trained Word2Vec models to compare languages. In this notebook, we'll use the `gensim` library to load a pre-trained Word2Vec model and compare the similarity of different words. The application for linear algebra is to experiment with cosine similarity, which measures the angle between two vectors. In this case, the result of cosine similarity will give us a measure of how similar two words are in meaning.

## Setup

First, you'll need to make sure you have all the necessary libraries installed. On canvas, you can find a file called `cse3380a4.yml` that lists all the libraries you'll need. You can install them by running the following command:

```bash
conda env create -f cse3380a4.yml
```

If that doesn't work for you, you can find a list of the libraries you'll need below:

- `gensim`
- `numpy`
- `matplotlib`
- `scipy==1.12.0`

To use the model, you'll need to download the pre-trained weights. You can find them [here](https://drive.google.com/file/d/19Z7UreEhC8Gm-PoWCWxXWSF0pFqgCUno/view?usp=sharing). Make sure you remember where you downloaded the file, as you'll need to provide the path to the model in the code below.

If everything is downloaded and installed, you should be able to run the cell below. Note that this may take a few minutes to load the model.

In [1]:
import gensim

model_file = 'GoogleNews-vectors-negative300.bin.gz' # path to the model file
model = gensim.models.KeyedVectors.load_word2vec_format(model_file, binary=True)

## Using the model

You can retrieve words from the model along with their embedded vectors. To do so, you can use the `key_to_index` attribute of the model. This will give you a dictionary where the keys are the words and the values are the corresponding vectors. An example is provided in the cell below.

The embedding is a 300-dimensional vector for each word.

In [2]:
# Retrieve a word and its embedding
result = model.index_to_key[1024]
embedding = model[result]
print(result, embedding)


vehicle [ 0.33789062 -0.05249023  0.08496094 -0.18847656 -0.05688477 -0.14355469
  0.16308594 -0.17285156  0.26367188 -0.12402344  0.16699219 -0.13867188
 -0.0135498  -0.0177002   0.03930664 -0.05712891 -0.15820312 -0.11572266
  0.15820312 -0.09667969  0.12695312 -0.30273438 -0.2421875   0.04638672
  0.11474609 -0.03515625 -0.04003906  0.32617188  0.32421875 -0.07275391
 -0.19628906 -0.25976562 -0.03393555  0.06689453 -0.10400391 -0.11914062
  0.15136719 -0.05249023 -0.16601562 -0.04418945 -0.03198242  0.11181641
 -0.07324219  0.24414062 -0.32226562  0.01141357  0.0189209   0.1640625
  0.265625    0.08642578 -0.31054688  0.16699219  0.16992188  0.15136719
 -0.13378906  0.07910156  0.07080078  0.21582031  0.1328125  -0.10400391
  0.08105469 -0.04516602 -0.35546875 -0.05639648 -0.13378906  0.27539062
 -0.02844238 -0.04907227 -0.03588867 -0.03125     0.01123047 -0.15917969
 -0.16601562  0.06445312 -0.05834961  0.05004883 -0.09960938 -0.08789062
 -0.00292969 -0.13378906  0.375       0.0006

## Comparing Vectors with Cosine Similarity

The cell below is where the primary work will be done. You'll need to compare the similarity of different words using cosine similarity. The formula for cosine similarity is given by:

$$
\text{cosine\_similarity}(\mathbf{a}, \mathbf{b}) = \frac{\mathbf{a} \cdot \mathbf{b}}{\|\mathbf{a}\| \|\mathbf{b}\| }
$$

where $\mathbf{a}$ and $\mathbf{b}$ are the vectors you want to compare. The dot product of the two vectors is divided by the product of their magnitudes.

You can use `numpy` and the `linalg` module to compute the dot product and magnitudes of the vectors. You should complete the function `cosine_similarity` below to compute the cosine similarity between two vectors.

In [3]:
import numpy as np

def similarity_search(word, similarity_func, model, topn=10):
    """Compare similarity to each word in the model.

    Args:
        word (str): The word to compare similarity to.
        similarity_func (function): The similarity function to use.
        model (gensim.models.KeyedVectors): The model to use.
        topn (int): The number of most similar words to return.

    Returns:
        list: The top n most similar words.
    """
    # Create a dictionary that will store the similarities
    # The keys are the words, the values are the similarities
    similarities = {}

    # Get the embedding for the search word
    word_embedding = model[word]

    # Compare the search word to each word in the model
    for word2 in model.key_to_index:
        word2_embedding = model[word2]

        # Skip the word itself
        if(word == word2): continue
        # Calculate the similarity, storing it in the dictionary associated with the word
        similarities[word2] = similarity_func(word_embedding, word2_embedding)

    # Sort by similarity
    most_similar = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

    # Return the top n most similar words
    return most_similar[:topn]


def cosine_similarity(word1_vec, word2_vec):
    """Returns cosine similarity between two vectors."""
    # TODO: Implement cosine similarity
    dot_product = np.dot(word1_vec, word2_vec)
    normal_w1 = np.linalg.norm(word1_vec)
    normal_w2 = np.linalg.norm(word2_vec)
    return dot_product/(normal_w1*normal_w2)


# Return the top 10 most similar words to "king"
print(similarity_search("cat", cosine_similarity, model))


[('cats', 0.809938), ('dog', 0.76094574), ('kitten', 0.7464985), ('feline', 0.73262346), ('beagle', 0.7150583), ('puppy', 0.70754534), ('pup', 0.6934291), ('pet', 0.68915313), ('felines', 0.6755932), ('chihuahua', 0.67097616)]


## Comparison with `gensim`'s similarity function

The cell below will compare the cosine similarity of your input word using the `gensim` library. You can use this as a reference to check your implementation.

In [4]:
# Return the top 10 most similar words to "king"
print(model.most_similar("cat", topn=10))

[('cats', 0.8099378347396851), ('dog', 0.7609457969665527), ('kitten', 0.7464984655380249), ('feline', 0.7326234579086304), ('beagle', 0.7150582671165466), ('puppy', 0.7075453996658325), ('pup', 0.6934291124343872), ('pet', 0.6891530752182007), ('felines', 0.6755931973457336), ('chihuahua', 0.6709762215614319)]
